In [11]:
##Creating folder
import os

# Create env folder if it doesn't exist
folder = "env"
if not os.path.exists(folder):
    os.makedirs(folder)
    print(f"Folder '{folder}' created.")
else:
    print(f"Folder '{folder}' already exists.")

# Verify write permission
test_file = os.path.join(folder, "test.txt")
with open(test_file, "w") as f:
    f.write("write test")
print("Write test succeeded, deleting test file.")
os.remove(test_file)


       

Folder 'env' already exists.
Write test succeeded, deleting test file.


In [12]:
env_code = """
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class HypertensionEnv(gym.Env):
    def __init__(self):
        super(HypertensionEnv, self).__init__()
        # Observation: age, BMI, systolic BP
        self.observation_space = spaces.Box(low=np.array([18,15,90]), high=np.array([80,45,200]), dtype=np.float32)
        # Action: 0 = No screen, 1 = Screen
        self.action_space = spaces.Discrete(2)
        # Rewards
        self.screen_cost = -1
        self.missed_case_penalty = -10
        self.detect_reward = 10
        self.reset()

    def reset(self, seed=None, options=None):
        self.state = self._generate_patient()
        return np.array(self.state, dtype=np.float32), {}

    def _generate_patient(self):
        age = np.random.randint(18,80)
        bmi = np.random.uniform(15,45)
        sbp = np.random.randint(90,200)
        return [age, bmi, sbp]

    def step(self, action):
        age, bmi, sbp = self.state
        has_hypertension = sbp >= 140
        if action == 1:
            reward = self.screen_cost
            if has_hypertension:
                reward += self.detect_reward
        else:
            reward = 0
            if has_hypertension:
                reward += self.missed_case_penalty
        self.state = self._generate_patient()
        return np.array(self.state, dtype=np.float32), reward, False, False, {}
"""

file_path = os.path.join("env", "hypertension_env.py")
with open(file_path, "w") as f:
    f.write(env_code)

print("Environment file created successfully!")
print("env folder contents:", os.listdir("env"))

Environment file created successfully!
env folder contents: ['hypertension_env.py']


In [13]:
#testing importing the environment
import sys
sys.path.append("env")  # allow Python to find env folder

from hypertension_env import HypertensionEnv

env = HypertensionEnv()
obs, _ = env.reset()
print("Environment loaded! Initial observation:", obs)

Environment loaded! Initial observation: [ 32.       43.76691 100.     ]


In [15]:
#train PPO
!pip install stable-baselines3 gymnasium --quiet

from stable_baselines3 import PPO

# Initialize PPO agent
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
model.learn(total_timesteps=50000)  # increase if time allows

# Save the trained model
model.save("ppo_hypertension_model")
print("Model saved as ppo_hypertension_model.zip")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1126 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 805         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016488642 |
|    clip_fraction        | 0.377       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.677      |
|    explained_variance   | -0.000352   |
|    learning_rate        | 0.0003      |
|    loss                 | 177         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0441     |
|    value_loss         

In [16]:
#test the trained model
# Load trained model
model = PPO.load("ppo_hypertension_model")

# Test prediction on a single patient
obs, _ = env.reset()
action, _ = model.predict(obs)
print("Observation:", obs)
print("Action:", "SCREEN" if action == 1 else "DON'T SCREEN")

Observation: [ 30.       32.91348 183.     ]
Action: SCREEN


In [22]:
##Deploying API
!pip install fastapi uvicorn nest-asyncio pyngrok --quiet

import nest_asyncio
import uvicorn
from uvicorn import Config, Server
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
from stable_baselines3 import PPO
from pyngrok import ngrok

# Load trained RL model

model = PPO.load("ppo_hypertension_model")  # Ensure this file exists

# Define FastAPI app
app = FastAPI(title="Hypertension Screening RL API")

class Patient(BaseModel):
    age: int
    bmi: float
    sbp: int

@app.post("/predict")
def predict(patient: Patient):
    obs = np.array([patient.age, patient.bmi, patient.sbp], dtype=np.float32)
    action, _ = model.predict(obs, deterministic=True)
    result = "SCREEN" if action == 1 else "DON'T SCREEN"
    return {"action": result}

# Apply nest_asyncio for Jupyter
nest_asyncio.apply()

# tart ngrok tunnel
ngrok.set_auth_token("35kJGOYtIITlFrBT6H381Dgk9aC_3vpwZ5aVroGB2bnY5y1jd")  
public_url = ngrok.connect(8000)
print("Public API URL:", public_url)

# Start FastAPI programmatically

config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = Server(config=config)

# Run server (non-blocking in Jupyter)
import threading
thread = threading.Thread(target=server.run)
thread.start()

Public API URL: NgrokTunnel: "https://billye-isentropic-seclusively.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [34324]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
